In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
air_data = pd.read_csv('./data/air_dict.csv')
test_data = pd.read_csv('./data/test_dict.csv')

In [3]:
air_data.head()

,air_store_id,visit_date,visitors,dom,dow,doy,month,quarter,year,store,holi,gen,area,doq
0,air_ba937bf13d40fb24,2016-01-13,25,12,2,12,0,0,0,517,0,13,33,12
1,air_ba937bf13d40fb24,2016-01-14,32,13,3,13,0,0,0,517,0,13,33,13
2,air_ba937bf13d40fb24,2016-01-15,29,14,4,14,0,0,0,517,0,13,33,14
3,air_ba937bf13d40fb24,2016-01-16,22,15,5,15,0,0,0,517,0,13,33,15
4,air_ba937bf13d40fb24,2016-01-18,6,17,0,17,0,0,0,517,0,13,33,17


In [4]:
air_data['id'] = np.ones(len(air_data))

In [5]:
together = pd.concat([air_data,test_data],axis=0)

In [6]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,quarter,store,visit_date,visitors,year
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,0,517,2016-01-13,25,0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,0,517,2016-01-14,32,0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,0,517,2016-01-15,29,0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,0,517,2016-01-16,22,0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,0,517,2016-01-18,6,0


In [7]:
test_data.head()

,id,visitors,air_store_id,visit_date,dom,dow,doy,month,quarter,year,store,holi,gen,area,doq
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,22,6,112,3,1,1,455,0,7,39,22
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,23,0,113,3,1,1,455,0,7,39,23
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,24,1,114,3,1,1,455,0,7,39,24
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,25,2,115,3,1,1,455,0,7,39,25
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,26,3,116,3,1,1,455,0,7,39,26


## stores of the same genre in the area

In [8]:
tmp = air_data.groupby(['gen','area'],as_index=False
                )['air_store_id'].count().rename(columns={'air_store_id':'area_genre_comp'})

In [9]:
air_data = pd.merge(left=air_data,right=tmp,on=['gen','area'])

In [10]:
tmp = air_data[['air_store_id','area_genre_comp']].drop_duplicates()
test_data = pd.merge(left=test_data,right=tmp,how='left',
                    on='air_store_id')

In [11]:
test_data.head()

,id,visitors,air_store_id,visit_date,dom,dow,doy,month,quarter,year,store,holi,gen,area,doq,area_genre_comp
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,22,6,112,3,1,1,455,0,7,39,22,938
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,23,0,113,3,1,1,455,0,7,39,23,938
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,24,1,114,3,1,1,455,0,7,39,24,938
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,25,2,115,3,1,1,455,0,7,39,25,938
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,26,3,116,3,1,1,455,0,7,39,26,938


## store count

In [12]:
tmp = air_data.groupby('area',
                       as_index=False)['air_store_id'].count().rename(columns={'air_store_id':'area_comp'})

In [13]:
air_data = pd.merge(air_data,tmp,how='inner',on='area')

In [14]:
tmp = air_data[['store','area_comp']].drop_duplicates()

In [15]:
test_data = pd.merge(left=test_data,right=tmp,how='left',on='store')

In [16]:
test_data.head()

,id,visitors,air_store_id,visit_date,dom,dow,doy,month,quarter,year,store,holi,gen,area,doq,area_genre_comp,area_comp
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,22,6,112,3,1,1,455,0,7,39,22,938,5411
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,23,0,113,3,1,1,455,0,7,39,23,938,5411
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,24,1,114,3,1,1,455,0,7,39,24,938,5411
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,25,2,115,3,1,1,455,0,7,39,25,938,5411
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,26,3,116,3,1,1,455,0,7,39,26,938,5411


## last quarter dow visitors

In [17]:
air_data.head()

,air_store_id,visit_date,visitors,dom,dow,doy,month,quarter,year,store,holi,gen,area,doq,id,area_genre_comp,area_comp
0,air_ba937bf13d40fb24,2016-01-13,25,12,2,12,0,0,0,517,0,13,33,12,1.0,2514,14696
1,air_ba937bf13d40fb24,2016-01-14,32,13,3,13,0,0,0,517,0,13,33,13,1.0,2514,14696
2,air_ba937bf13d40fb24,2016-01-15,29,14,4,14,0,0,0,517,0,13,33,14,1.0,2514,14696
3,air_ba937bf13d40fb24,2016-01-16,22,15,5,15,0,0,0,517,0,13,33,15,1.0,2514,14696
4,air_ba937bf13d40fb24,2016-01-18,6,17,0,17,0,0,0,517,0,13,33,17,1.0,2514,14696


In [18]:
tmp = together.groupby(['year','quarter','gen','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'quarterly_genre_visit'})

In [19]:
tmp.head()

,year,quarter,gen,area,quarterly_genre_visit
0,0,0,0,0,2638
1,0,0,0,1,1488
2,0,0,0,3,2210
3,0,0,0,4,809
4,0,0,0,5,2822


In [20]:
tmp_no_gen = together.groupby(['year','quarter','area'],as_index=False
                             )['visitors'].sum().rename(columns={'visitors':'quarterly_visit'})

In [21]:
tmp_no_gen.shape

(584, 4)

In [22]:
tmp = pd.merge(tmp,tmp_no_gen,how='inner',on=['year','quarter','area'])

In [23]:
tmp.head()

,year,quarter,gen,area,quarterly_genre_visit,quarterly_visit
0,0,0,0,0,2638,2638
1,0,0,0,1,1488,11183
2,0,0,1,1,1422,11183
3,0,0,5,1,1601,11183
4,0,0,6,1,6672,11183


In [24]:
tmp.shape

(1331, 6)

In [25]:
tmp.quarterly_visit.any(0)

True

In [26]:
def get_gen_perc(data):
    if data.quarterly_visit == 0:
        return 0
    else:
        return data.quarterly_genre_visit/data.quarterly_visit

In [27]:
tmp.iloc[:3].apply(get_gen_perc,axis=1)

0    1.000000
1    0.133059
2    0.127157
dtype: float64

In [28]:
tmp['gen_pop_quarterly'] = tmp.apply(get_gen_perc,axis=1)

In [29]:
tmp.head()

,year,quarter,gen,area,quarterly_genre_visit,quarterly_visit,gen_pop_quarterly
0,0,0,0,0,2638,2638,1.000000
1,0,0,0,1,1488,11183,0.133059
2,0,0,1,1,1422,11183,0.127157
3,0,0,5,1,1601,11183,0.143164
4,0,0,6,1,6672,11183,0.596620


In [30]:
tmp = tmp.sort_values(by=['gen','area','year','quarter'])

In [31]:
def get_gen_area(data):
    return str(data.gen)+str(data.area)

In [32]:
tmp.iloc[:3].apply(get_gen_area,axis=1)

0      0.00.0
177    0.00.0
356    0.00.0
dtype: object

In [33]:
tmp['gen_area'] = tmp.apply(get_gen_area,axis=1)

In [34]:
tmp.head()

,year,quarter,gen,area,quarterly_genre_visit,quarterly_visit,gen_pop_quarterly,gen_area
0,0,0,0,0,2638,2638,1.0,0.00.0
177,0,1,0,0,2521,2521,1.0,0.00.0
356,0,2,0,0,2623,2623,1.0,0.00.0
600,0,3,0,0,2928,2928,1.0,0.00.0
844,1,0,0,0,2553,2553,1.0,0.00.0


In [35]:
tmp[['gen_area','year','quarter','gen_pop_quarterly']].head()

,gen_area,year,quarter,gen_pop_quarterly
0,0.00.0,0,0,1.0
177,0.00.0,0,1,1.0
356,0.00.0,0,2,1.0
600,0.00.0,0,3,1.0
844,0.00.0,1,0,1.0


In [36]:
tmp['last_gen_pop_quarterly']=tmp.gen_pop_quarterly.shift(1)

In [37]:
tmp.head()

,year,quarter,gen,area,quarterly_genre_visit,quarterly_visit,gen_pop_quarterly,gen_area,last_gen_pop_quarterly
0,0,0,0,0,2638,2638,1.0,0.00.0,NaN
177,0,1,0,0,2521,2521,1.0,0.00.0,1.0
356,0,2,0,0,2623,2623,1.0,0.00.0,1.0
600,0,3,0,0,2928,2928,1.0,0.00.0,1.0
844,1,0,0,0,2553,2553,1.0,0.00.0,1.0


In [38]:
tmp['gen_area_shift'] = tmp.gen_area.shift(1)

In [39]:
def get_lqpq(data):
    if data.gen_area_shift!=data.gen_area:
        return -1
    else:
        return data.last_gen_pop_quarterly

In [40]:
tmp['last_quarter_gen_pop']=tmp.apply(get_lqpq,axis=1)

In [41]:
tmp.head()

,year,quarter,gen,area,quarterly_genre_visit,quarterly_visit,gen_pop_quarterly,gen_area,last_gen_pop_quarterly,gen_area_shift,last_quarter_gen_pop
0,0,0,0,0,2638,2638,1.0,0.00.0,NaN,NaN,-1.0
177,0,1,0,0,2521,2521,1.0,0.00.0,1.0,0.00.0,1.0
356,0,2,0,0,2623,2623,1.0,0.00.0,1.0,0.00.0,1.0
600,0,3,0,0,2928,2928,1.0,0.00.0,1.0,0.00.0,1.0
844,1,0,0,0,2553,2553,1.0,0.00.0,1.0,0.00.0,1.0


In [42]:
tmp = tmp[['year','quarter','gen','area','last_quarter_gen_pop']]

In [43]:
together = pd.merge(together,tmp,how='inner',on=['year','quarter','gen','area'])

In [44]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,quarter,store,visit_date,visitors,year,last_quarter_gen_pop
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,0,517,2016-01-13,25,0,-1.0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,0,517,2016-01-14,32,0,-1.0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,0,517,2016-01-15,29,0,-1.0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,0,517,2016-01-16,22,0,-1.0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,0,517,2016-01-18,6,0,-1.0


## last quarter store pop in area

In [45]:
tmp = together.groupby(['year','quarter','store','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'store_visit_by_area'})

In [46]:
tmp0 = together.groupby(['year','quarter','area'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'area_visit'})

In [47]:
tmp = pd.merge(tmp,tmp0,how='inner',on=['year','quarter','area'])

In [48]:
tmp.head()

,year,quarter,store,area,store_visit_by_area,area_visit
0,0,0,3,6,3733,7505
1,0,0,392,6,2576,7505
2,0,0,573,6,1196,7505
3,0,0,6,19,1052,7881
4,0,0,79,19,2723,7881


In [49]:
def store_pop(data):
    if data.area_visit==0:
        return 0 
    else:
        return data.store_visit_by_area/data.area_visit

In [50]:
tmp.iloc[:3].apply(store_pop,axis=1)

0    0.497402
1    0.343238
2    0.159360
dtype: float64

In [51]:
tmp['store_pop'] = tmp.apply(store_pop,axis=1)

In [52]:
tmp.head()

,year,quarter,store,area,store_visit_by_area,area_visit,store_pop
0,0,0,3,6,3733,7505,0.497402
1,0,0,392,6,2576,7505,0.343238
2,0,0,573,6,1196,7505,0.159360
3,0,0,6,19,1052,7881,0.133486
4,0,0,79,19,2723,7881,0.345515


In [53]:
tmp = tmp.sort_values(by=['store','area','year','quarter'])

In [54]:
tmp['last_quarter_store_pop'] = tmp.store_pop.shift(1)

In [55]:
tmp.head(20)

,year,quarter,store,area,store_visit_by_area,area_visit,store_pop,last_quarter_store_pop
627,0,2,0,64,1469,4354,0.337391,NaN
1438,0,3,0,64,1653,4383,0.377139,0.337391
2255,1,0,0,64,2071,4467,0.463622,0.377139
3080,1,1,0,64,662,1421,0.465869,0.463622
629,0,2,1,58,1032,53903,0.019146,0.465869
1440,0,3,1,58,930,60815,0.015292,0.019146
2257,1,0,1,58,775,56224,0.013784,0.015292
3082,1,1,1,58,224,14917,0.015016,0.013784
668,0,2,2,19,1348,25016,0.053886,0.015016
1479,0,3,2,19,1159,29769,0.038933,0.053886


In [56]:
tmp['store_area'] = tmp.apply(lambda x:str(x.store)+str(x.area),axis=1)

In [57]:
tmp.head()

,year,quarter,store,area,store_visit_by_area,area_visit,store_pop,last_quarter_store_pop,store_area
627,0,2,0,64,1469,4354,0.337391,NaN,0.064.0
1438,0,3,0,64,1653,4383,0.377139,0.337391,0.064.0
2255,1,0,0,64,2071,4467,0.463622,0.377139,0.064.0
3080,1,1,0,64,662,1421,0.465869,0.463622,0.064.0
629,0,2,1,58,1032,53903,0.019146,0.465869,1.058.0


In [58]:
tmp['store_area_shift'] = tmp.store_area.shift(1)

In [59]:
def get_lqsp(data):
    if data.store_area_shift!=data.store_area:
        return -1
    else:
        return data.last_quarter_store_pop

In [60]:
tmp['last_quarter_s_pop'] = tmp.apply(get_lqsp,axis=1)

In [61]:
tmp.head()

,year,quarter,store,area,store_visit_by_area,area_visit,store_pop,last_quarter_store_pop,store_area,store_area_shift,last_quarter_s_pop
627,0,2,0,64,1469,4354,0.337391,NaN,0.064.0,NaN,-1.000000
1438,0,3,0,64,1653,4383,0.377139,0.337391,0.064.0,0.064.0,0.337391
2255,1,0,0,64,2071,4467,0.463622,0.377139,0.064.0,0.064.0,0.377139
3080,1,1,0,64,662,1421,0.465869,0.463622,0.064.0,0.064.0,0.463622
629,0,2,1,58,1032,53903,0.019146,0.465869,1.058.0,0.064.0,-1.000000


In [62]:
tmp = tmp[['year','quarter','store','area','last_quarter_s_pop']]

In [63]:
together = pd.merge(together,tmp,how='inner',on=['year','quarter','store','area'])

In [64]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,quarter,store,visit_date,visitors,year,last_quarter_gen_pop,last_quarter_s_pop
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,0,517,2016-01-13,25,0,-1.0,-1.0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,0,517,2016-01-14,32,0,-1.0,-1.0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,0,517,2016-01-15,29,0,-1.0,-1.0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,0,517,2016-01-16,22,0,-1.0,-1.0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,0,517,2016-01-18,6,0,-1.0,-1.0


## store pop in each area overall

In [65]:
tmp = together.groupby(['air_store_id','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'store_visitors'})

In [66]:
tmp.head()

,air_store_id,area,store_visitors
0,air_00a91d42b08b08d9,39,6051
1,air_0164b9927d20bcc3,33,1378
2,air_0241aa3964b7f861,1,3919
3,air_0328696196e46f18,96,921
4,air_034a3d5b40d5b1b1,84,3722


In [67]:
tmp0 = together.groupby(['area'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'area_total_visitors'})

In [68]:
tmp0.head()

,area,area_total_visitors
0,0,13836
1,1,120727
2,2,14468
3,3,19723
4,4,5962


In [69]:
tmp = pd.merge(tmp,tmp0,how='inner',on='area')

In [70]:
tmp.head()

,air_store_id,area,store_visitors,area_total_visitors
0,air_00a91d42b08b08d9,39,6051,143381
1,air_2c989829acbd1c6b,39,10493,143381
2,air_36bcf77d3382d36e,39,15126,143381
3,air_47beaffd3806c979,39,3780,143381
4,air_5bd22f9cc1426a90,39,6360,143381


In [71]:
tmp0 = together.groupby(['gen','area'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'area_gen_total_visitors'})

In [72]:
tmp0.head()

,gen,area,area_gen_total_visitors
0,0,0,13836
1,0,1,18544
2,0,2,14468
3,0,3,19723
4,0,4,5962


In [73]:
tmp0 = pd.merge(tmp0,together[['air_store_id','gen','area']].drop_duplicates(),how='right',on=['gen','area'])

In [74]:
tmp0.shape

(829, 4)

In [75]:
tmp0.head()

,gen,area,area_gen_total_visitors,air_store_id
0,0,0,13836,air_a563896da3777078
1,0,0,13836,air_7d65049f9d275c0d
2,0,1,18544,air_b2d97bd2337c5ba7
3,0,1,18544,air_0241aa3964b7f861
4,0,1,18544,air_f068442ebb6c246c


In [76]:
tmp = pd.merge(tmp,tmp0[['air_store_id','area_gen_total_visitors']],how='inner',on='air_store_id')

In [77]:
tmp.head()

,air_store_id,area,store_visitors,area_total_visitors,area_gen_total_visitors
0,air_00a91d42b08b08d9,39,6051,143381,21876
1,air_2c989829acbd1c6b,39,10493,143381,55935
2,air_36bcf77d3382d36e,39,15126,143381,19647
3,air_47beaffd3806c979,39,3780,143381,12656
4,air_5bd22f9cc1426a90,39,6360,143381,12235


In [78]:
tmp.shape

(829, 5)

In [79]:
tmp.area_total_visitors.any(0)

True

In [80]:
def store_over_area(data):
    if data.area_total_visitors == 0:
        return 0
    else:
        return data.store_visitors/data.area_total_visitors
def store_over_gen(data):
    if data.area_gen_total_visitors == 0:
        return 0
    else:
        return data.store_visitors/data.area_gen_total_visitors
def gen_over_area(data):
    if data.area_total_visitors == 0:
        return 0
    else:
        return data.area_gen_total_visitors/data.area_total_visitors

In [81]:
tmp.drop('area',axis=1,inplace=True)

In [82]:
tmp.head()

,air_store_id,store_visitors,area_total_visitors,area_gen_total_visitors
0,air_00a91d42b08b08d9,6051,143381,21876
1,air_2c989829acbd1c6b,10493,143381,55935
2,air_36bcf77d3382d36e,15126,143381,19647
3,air_47beaffd3806c979,3780,143381,12656
4,air_5bd22f9cc1426a90,6360,143381,12235


In [83]:
tmp['store_pop'] = tmp.apply(store_over_area,axis=1)

In [84]:
tmp['store_genre_pop'] = tmp.apply(store_over_gen,axis=1)

In [85]:
tmp['genre_pop'] = tmp.apply(gen_over_area,axis=1)

In [86]:
tmp.head()

,air_store_id,store_visitors,area_total_visitors,area_gen_total_visitors,store_pop,store_genre_pop,genre_pop
0,air_00a91d42b08b08d9,6051,143381,21876,0.042202,0.276604,0.152573
1,air_2c989829acbd1c6b,10493,143381,55935,0.073183,0.187593,0.390114
2,air_36bcf77d3382d36e,15126,143381,19647,0.105495,0.769889,0.137027
3,air_47beaffd3806c979,3780,143381,12656,0.026363,0.298673,0.088268
4,air_5bd22f9cc1426a90,6360,143381,12235,0.044357,0.519820,0.085332


In [87]:
tmp = tmp[['air_store_id','store_pop','store_genre_pop','genre_pop']]

In [88]:
together = pd.merge(together,tmp,how='inner',on='air_store_id')

In [89]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,quarter,store,visit_date,visitors,year,last_quarter_gen_pop,last_quarter_s_pop,store_pop,store_genre_pop,genre_pop
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,0,517,2016-01-13,25,0,-1.0,-1.0,0.030911,0.176627,0.175009
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,0,517,2016-01-14,32,0,-1.0,-1.0,0.030911,0.176627,0.175009
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,0,517,2016-01-15,29,0,-1.0,-1.0,0.030911,0.176627,0.175009
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,0,517,2016-01-16,22,0,-1.0,-1.0,0.030911,0.176627,0.175009
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,0,517,2016-01-18,6,0,-1.0,-1.0,0.030911,0.176627,0.175009


In [90]:
together.columns

Index(['air_store_id', 'area', 'dom', 'doq', 'dow', 'doy', 'gen', 'holi', 'id',
       'month', 'quarter', 'store', 'visit_date', 'visitors', 'year',
       'last_quarter_gen_pop', 'last_quarter_s_pop', 'store_pop',
       'store_genre_pop', 'genre_pop'],
      dtype='object')

## split for train just to test

In [91]:
train_id = together.id.apply(lambda x:True if x==1.0 else False)

In [92]:
train_data = together[train_id]

In [93]:
train_data.shape

(252108, 20)

In [94]:
together.shape

(284127, 20)

In [95]:
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.534389
0.15 0.517530666667
0.2 0.516043333333


## continue with together data

## last month genre and store pop

In [96]:
tmp = together.groupby(['year','month','store','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'store_visit_by_area'})

In [97]:
tmp.head()

,year,month,store,area,store_visit_by_area
0,0,0,3,6,978
1,0,0,6,19,322
2,0,0,8,48,195
3,0,0,11,33,202
4,0,0,12,1,356


In [98]:
tmp0 = together.groupby(['year','month','area'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'area_total_visit'})

In [99]:
tmp0.head()

,year,month,area,area_total_visit
0,0,0,0,860
1,0,0,1,3461
2,0,0,3,733
3,0,0,4,252
4,0,0,5,1272


In [100]:
tmp.shape

(10795, 5)

In [101]:
tmp = pd.merge(tmp,tmp0,how='inner',on=['year','month','area'])

In [102]:
tmp.shape

(10795, 6)

In [103]:
tmp.head()

,year,month,store,area,store_visit_by_area,area_total_visit
0,0,0,3,6,978,2084
1,0,0,392,6,713,2084
2,0,0,573,6,393,2084
3,0,0,6,19,322,2310
4,0,0,79,19,838,2310


In [104]:
def get_lmsp(data):
    if data.area_total_visit==0:
        return 0
    else:
        return data.store_visit_by_area/data.area_total_visit

In [105]:
tmp.iloc[:3].apply(get_lmsp,axis=1)

0    0.469290
1    0.342131
2    0.188580
dtype: float64

In [106]:
tmp = tmp.sort_values(by=['store','area','year','month'])

In [107]:
tmp.head(20)

,year,month,store,area,store_visit_by_area,area_total_visit
1856,0,6,0,64,430,1449
2655,0,7,0,64,537,1459
3457,0,8,0,64,502,1446
4264,0,9,0,64,521,1592
5073,0,10,0,64,554,1333
5884,0,11,0,64,578,1458
6695,1,0,0,64,562,1426
7509,1,1,0,64,511,1188
8325,1,2,0,64,998,1853
9150,1,3,0,64,662,1421


In [108]:
tmp['store_pop'] = tmp.apply(get_lmsp,axis=1)

In [109]:
tmp['store_pop_shift'] = tmp.store_pop.shift(1)

In [110]:
tmp.head()

,year,month,store,area,store_visit_by_area,area_total_visit,store_pop,store_pop_shift
1856,0,6,0,64,430,1449,0.296756,NaN
2655,0,7,0,64,537,1459,0.368060,0.296756
3457,0,8,0,64,502,1446,0.347165,0.368060
4264,0,9,0,64,521,1592,0.327261,0.347165
5073,0,10,0,64,554,1333,0.415604,0.327261


In [111]:
tmp['store_shift'] = tmp.store.shift(1)

In [112]:
def get_first_lmsp(data):
    if data.store!=data.store_shift:
        return -1
    else:
        return data.store_pop_shift

In [113]:
tmp['store_pop_monthly'] = tmp.apply(get_first_lmsp,axis=1)

In [114]:
tmp.head()

,year,month,store,area,store_visit_by_area,area_total_visit,store_pop,store_pop_shift,store_shift,store_pop_monthly
1856,0,6,0,64,430,1449,0.296756,NaN,NaN,-1.000000
2655,0,7,0,64,537,1459,0.368060,0.296756,0.0,0.296756
3457,0,8,0,64,502,1446,0.347165,0.368060,0.0,0.368060
4264,0,9,0,64,521,1592,0.327261,0.347165,0.0,0.347165
5073,0,10,0,64,554,1333,0.415604,0.327261,0.0,0.327261


In [115]:
tmp = tmp[['year','month','store','area','store_pop_monthly']]

In [116]:
tmp.head()

,year,month,store,area,store_pop_monthly
1856,0,6,0,64,-1.000000
2655,0,7,0,64,0.296756
3457,0,8,0,64,0.368060
4264,0,9,0,64,0.347165
5073,0,10,0,64,0.327261


In [117]:
together = pd.merge(together,tmp,how='inner',on=['year','month','store','area'])

In [118]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,...,store,visit_date,visitors,year,last_quarter_gen_pop,last_quarter_s_pop,store_pop,store_genre_pop,genre_pop,store_pop_monthly
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,...,517,2016-01-13,25,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,...,517,2016-01-14,32,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,...,517,2016-01-15,29,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,...,517,2016-01-16,22,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,...,517,2016-01-18,6,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0


In [119]:
tmp.tail()

,year,month,store,area,store_pop_monthly
6804,1,0,828,39,0.015176
7618,1,1,828,39,0.018061
8434,1,2,828,39,0.013227
9259,1,3,828,39,0.013591
10083,1,4,828,39,0.011050


## monthly genre pop in each area

In [120]:
tmp = together.groupby(['year','month','gen','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'genre_visit_by_area'})

In [121]:
tmp.shape

(3735, 5)

In [122]:
tmp0.shape

(1645, 4)

In [123]:
tmp0.head()

,year,month,area,area_total_visit
0,0,0,0,860
1,0,0,1,3461
2,0,0,3,733
3,0,0,4,252
4,0,0,5,1272


In [124]:
tmp = pd.merge(tmp,tmp0,how='inner',on=['year','month','area'])

In [125]:
tmp.shape

(3735, 6)

In [126]:
tmp.head()

,year,month,gen,area,genre_visit_by_area,area_total_visit
0,0,0,0,0,860,860
1,0,0,0,1,480,3461
2,0,0,1,1,512,3461
3,0,0,5,1,506,3461
4,0,0,6,1,1963,3461


In [127]:
def get_pop(data):
    if data.area_total_visit==0:
        return 0
    else:
        return data.genre_visit_by_area/data.area_total_visit

In [128]:
tmp['genre_pop'] = tmp.apply(get_pop,axis=1)

In [129]:
tmp.head()

,year,month,gen,area,genre_visit_by_area,area_total_visit,genre_pop
0,0,0,0,0,860,860,1.000000
1,0,0,0,1,480,3461,0.138688
2,0,0,1,1,512,3461,0.147934
3,0,0,5,1,506,3461,0.146201
4,0,0,6,1,1963,3461,0.567177


In [130]:
tmp = tmp.sort_values(by=['gen','area','year','month'])

In [131]:
tmp.head(10)

,year,month,gen,area,genre_visit_by_area,area_total_visit,genre_pop
0,0,0,0,0,860,860,1.0
172,0,1,0,0,740,740,1.0
346,0,2,0,0,1038,1038,1.0
520,0,3,0,0,878,878,1.0
698,0,4,0,0,797,797,1.0
877,0,5,0,0,846,846,1.0
1056,0,6,0,0,981,981,1.0
1298,0,7,0,0,729,729,1.0
1541,0,8,0,0,913,913,1.0
1785,0,9,0,0,925,925,1.0


In [132]:
tmp['gen_shift'] = tmp.gen.shift(1)
tmp['genre_pop'] = tmp.genre_pop.shift(1)

In [133]:
def get_lmgp(data):
    if data.gen!=data.gen_shift:
        return -1
    else:
        return data.genre_pop

In [134]:
tmp['last_genre_pop'] = tmp.apply(get_lmgp,axis=1)

In [135]:
tmp.head()

,year,month,gen,area,genre_visit_by_area,area_total_visit,genre_pop,gen_shift,last_genre_pop
0,0,0,0,0,860,860,NaN,NaN,-1.0
172,0,1,0,0,740,740,1.0,0.0,1.0
346,0,2,0,0,1038,1038,1.0,0.0,1.0
520,0,3,0,0,878,878,1.0,0.0,1.0
698,0,4,0,0,797,797,1.0,0.0,1.0


In [136]:
tmp = tmp[['year','month','gen','area','last_genre_pop']]

In [137]:
tmp.head()

,year,month,gen,area,last_genre_pop
0,0,0,0,0,-1.0
172,0,1,0,0,1.0
346,0,2,0,0,1.0
520,0,3,0,0,1.0
698,0,4,0,0,1.0


In [138]:
together = pd.merge(together,tmp,how='inner',on=['year','month','gen','area'])

In [139]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,...,visit_date,visitors,year,last_quarter_gen_pop,last_quarter_s_pop,store_pop,store_genre_pop,genre_pop,store_pop_monthly,last_genre_pop
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,...,2016-01-13,25,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,...,2016-01-14,32,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,...,2016-01-15,29,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,...,2016-01-16,22,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,...,2016-01-18,6,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0


## another round of xgb

In [140]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.518909666667
0.15 0.500968666667
0.2 0.498968333333


## back to together

In [141]:
tmp = together.groupby(['year','month','store','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'store_visit_by_area'})

In [142]:
tmp0 = together.groupby(['year','month','gen','area'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'gen_visit_by_area'})

In [143]:
tmp.shape

(10795, 5)

In [144]:
tmp0.shape

(3735, 5)

In [145]:
tmp = pd.merge(left=tmp,right=together[['store','gen']].drop_duplicates(),how='inner',on=['store'])

In [146]:
tmp.shape

(10795, 6)

In [147]:
tmp = pd.merge(left=tmp,right=tmp0,how='left',on=['year','month','area','gen'])

In [148]:
tmp.shape

(10795, 7)

In [149]:
tmp.head()

,year,month,store,area,store_visit_by_area,gen,gen_visit_by_area
0,0,0,3,6,978,6,978
1,0,1,3,6,1211,6,1211
2,0,2,3,6,1544,6,1544
3,0,3,3,6,1481,6,1481
4,0,4,3,6,1685,6,1685


In [150]:
tmp = tmp.sort_values(by=['store','area','year','month'])

In [151]:
tmp.tail()

,year,month,store,area,store_visit_by_area,gen,gen_visit_by_area
10595,1,0,828,39,182,1,246
10596,1,1,828,39,161,1,232
10597,1,2,828,39,210,1,270
10598,1,3,828,39,123,1,177
10599,1,4,828,39,0,1,0


In [152]:
def get_store_comp(data):
    if data.gen_visit_by_area==0:
        return 0
    else:
        return data.store_visit_by_area/data.gen_visit_by_area

In [153]:
tmp['gen_pop']=tmp.apply(get_store_comp,axis=1)

In [154]:
tmp['store_shift'] = tmp.store.shift(1)
tmp['gen_pop'] = tmp.gen_pop.shift(1)

In [155]:
tmp.head()

,year,month,store,area,store_visit_by_area,gen,gen_visit_by_area,gen_pop,store_shift
5301,0,6,0,64,430,6,1449,NaN,NaN
5302,0,7,0,64,537,6,1459,0.296756,0.0
5303,0,8,0,64,502,6,1446,0.368060,0.0
5304,0,9,0,64,521,6,1592,0.347165,0.0
5305,0,10,0,64,554,6,1333,0.327261,0.0


In [156]:
def get_lmc(data):
    if data.store!=data.store_shift:
        return -1
    else:
        return data.gen_pop

In [157]:
tmp['last_month_comp'] = tmp.apply(get_lmc,axis=1)

In [158]:
tmp.head()

,year,month,store,area,store_visit_by_area,gen,gen_visit_by_area,gen_pop,store_shift,last_month_comp
5301,0,6,0,64,430,6,1449,NaN,NaN,-1.000000
5302,0,7,0,64,537,6,1459,0.296756,0.0,0.296756
5303,0,8,0,64,502,6,1446,0.368060,0.0,0.368060
5304,0,9,0,64,521,6,1592,0.347165,0.0,0.347165
5305,0,10,0,64,554,6,1333,0.327261,0.0,0.327261


In [159]:
tmp = tmp[['year','month','store','area','last_month_comp']]

In [160]:
tmp.head()

,year,month,store,area,last_month_comp
5301,0,6,0,64,-1.000000
5302,0,7,0,64,0.296756
5303,0,8,0,64,0.368060
5304,0,9,0,64,0.347165
5305,0,10,0,64,0.327261


In [161]:
tmp.shape

(10795, 5)

In [162]:
together = pd.merge(together,tmp,how='inner',on=['year','month','store','area'])

In [163]:
together.shape

(284127, 23)

In [164]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,...,visitors,year,last_quarter_gen_pop,last_quarter_s_pop,store_pop,store_genre_pop,genre_pop,store_pop_monthly,last_genre_pop,last_month_comp
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,...,25,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,...,32,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,...,29,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,...,22,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,...,6,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0


## last quarter comp

In [165]:
tmp = together.groupby(['year','quarter','store','area'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'store_visit_by_area'})

In [166]:
tmp = pd.merge(tmp,together[['store','gen']].drop_duplicates(),how='inner',on='store')

In [167]:
tmp.shape

(3904, 6)

In [168]:
tmp0 = together.groupby(['year','quarter','gen','area'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'gen_visit_by_area'})

In [169]:
tmp0.shape

(1331, 5)

In [170]:
tmp = pd.merge(tmp,tmp0,how='inner',on=['year','quarter','gen','area'])

In [171]:
tmp.shape

(3904, 7)

In [172]:
tmp = tmp.sort_values(by=['store','area','year','quarter'])

In [173]:
tmp.head(10)

,year,quarter,store,area,store_visit_by_area,gen,gen_visit_by_area
2170,0,2,0,64,1469,6,4354
2172,0,3,0,64,1653,6,4383
2174,1,0,0,64,2071,6,4467
2176,1,1,0,64,662,6,1421
131,0,2,1,58,1032,13,9719
140,0,3,1,58,930,13,9298
149,1,0,1,58,775,13,8441
158,1,1,1,58,224,13,2254
640,0,2,2,19,1348,0,16728
648,0,3,2,19,1159,0,20261


In [174]:
def get_quarter_comp(data):
    if data.gen_visit_by_area==0:
        return 0
    else:
        return data.store_visit_by_area/data.gen_visit_by_area

In [175]:
tmp['quarter_comp'] = tmp.apply(get_quarter_comp,axis=1)

In [176]:
tmp['store_shift'] = tmp.store.shift(1)
tmp['quarter_comp'] = tmp.quarter_comp.shift(1)

In [177]:
tmp.head()

,year,quarter,store,area,store_visit_by_area,gen,gen_visit_by_area,quarter_comp,store_shift
2170,0,2,0,64,1469,6,4354,NaN,NaN
2172,0,3,0,64,1653,6,4383,0.337391,0.0
2174,1,0,0,64,2071,6,4467,0.377139,0.0
2176,1,1,0,64,662,6,1421,0.463622,0.0
131,0,2,1,58,1032,13,9719,0.465869,0.0


In [178]:
def get_lqc(data):
    if data.store!=data.store_shift:
        return -1
    else:
        return data.quarter_comp

In [179]:
tmp['last_quarter_comp'] = tmp.apply(get_lqc,axis=1)

In [180]:
tmp.head()

,year,quarter,store,area,store_visit_by_area,gen,gen_visit_by_area,quarter_comp,store_shift,last_quarter_comp
2170,0,2,0,64,1469,6,4354,NaN,NaN,-1.000000
2172,0,3,0,64,1653,6,4383,0.337391,0.0,0.337391
2174,1,0,0,64,2071,6,4467,0.377139,0.0,0.377139
2176,1,1,0,64,662,6,1421,0.463622,0.0,0.463622
131,0,2,1,58,1032,13,9719,0.465869,0.0,-1.000000


In [181]:
tmp = tmp[['year','quarter','store','area','last_quarter_comp']]

In [182]:
together = pd.merge(together,tmp,how='inner',on=['year','quarter','store','area'])

In [183]:
together.shape

(284127, 24)

In [184]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,...,year,last_quarter_gen_pop,last_quarter_s_pop,store_pop,store_genre_pop,genre_pop,store_pop_monthly,last_genre_pop,last_month_comp,last_quarter_comp
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,...,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,...,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,...,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,...,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,...,0,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0


## another xgb check

In [185]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.520493
0.15 0.501567
0.2 0.500811


In [186]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date','last_quarter_comp'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.520642333333
0.15 0.503113
0.2 0.501623


In [187]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date','last_month_comp',
                                     'last_quarter_comp','store_pop_monthly'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.518506333333
0.15 0.500707
0.2 0.498888333333


## back to together, area daily pattern

In [188]:
tmp = train_data.groupby(['store','dow'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'store_dow_visits'})

In [189]:
tmp.shape

(5773, 3)

In [190]:
tmp0 = train_data.groupby(['store'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'store_yearly_visits'})

In [191]:
tmp0.shape

(829, 2)

In [192]:
tmp = pd.merge(tmp,tmp0,how='inner',on=['store'])

In [193]:
tmp.shape

(5773, 4)

In [194]:
tmp = tmp.sort_values(by=['store','dow'])

In [195]:
tmp.head(10)

,store,dow,store_dow_visits,store_yearly_visits
0,0,0,140,5791
1,0,1,789,5791
2,0,2,765,5791
3,0,3,857,5791
4,0,4,816,5791
5,0,5,1198,5791
6,0,6,1226,5791
7,1,0,269,2780
8,1,1,390,2780
9,1,2,420,2780


In [196]:
tmp.store_yearly_visits.any(0)

True

In [197]:
def get_daily(data):
    if data.store_yearly_visits==0:
        return 0
    else:
        return data.store_dow_visits/data.store_yearly_visits

In [198]:
tmp.iloc[:3].apply(get_daily,axis=1)

0    0.024175
1    0.136246
2    0.132102
dtype: float64

In [199]:
tmp['dow_pattern_2016'] = tmp.apply(get_daily,axis=1)

In [200]:
tmp.head()

,store,dow,store_dow_visits,store_yearly_visits,dow_pattern_2016
0,0,0,140,5791,0.024175
1,0,1,789,5791,0.136246
2,0,2,765,5791,0.132102
3,0,3,857,5791,0.147988
4,0,4,816,5791,0.140908


In [201]:
tmp = tmp[['store','dow','dow_pattern_2016']]

In [202]:
together = pd.merge(together,tmp,how='left',on=['store','dow'])

In [203]:
together.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,...,last_quarter_gen_pop,last_quarter_s_pop,store_pop,store_genre_pop,genre_pop,store_pop_monthly,last_genre_pop,last_month_comp,last_quarter_comp,dow_pattern_2016
0,air_ba937bf13d40fb24,33,12,12,2,12,13,0,1,0,...,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0,0.171307
1,air_ba937bf13d40fb24,33,13,13,3,13,13,0,1,0,...,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0,0.148069
2,air_ba937bf13d40fb24,33,14,14,4,14,13,0,1,0,...,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0,0.253480
3,air_ba937bf13d40fb24,33,15,15,5,15,13,0,1,0,...,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0,0.204872
4,air_ba937bf13d40fb24,33,17,17,0,17,13,0,1,0,...,-1.0,-1.0,0.030911,0.176627,0.175009,-1.0,0.0,-1.0,-1.0,0.088011


In [204]:
together.shape

(284127, 25)

In [207]:
together.fillna(-1,inplace=True)

## another round of xgb test

In [208]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.502043666667
0.15 0.486849333333
0.2 0.487545


## day of month visit

In [239]:
tmp = together[together.year==0].groupby(['store','dom'],as_index=False
                      )['visitors'].sum().rename(columns={'visitors':'dom_visits'})

In [240]:
tmp0 = together[together.year==0].groupby(['store'],as_index=False
                       )['visitors'].sum().rename(columns={'visitors':'mon_visitors'})

In [241]:
tmp.shape

(25231, 3)

In [242]:
tmp0.shape

(819, 2)

In [243]:
tmp = pd.merge(tmp,tmp0,how='inner',on='store')

In [244]:
tmp.shape

(25231, 4)

In [245]:
tmp.head()

,store,dom,dom_visits,mon_visitors
0,0,0,84,3122
1,0,1,113,3122
2,0,2,118,3122
3,0,3,136,3122
4,0,4,87,3122


In [246]:
def get_dom_visit(data):
    if data.mon_visitors==0:
        return 0
    else:
        return data.dom_visits/data.mon_visitors

In [247]:
tmp['dom_pattern_train'] = tmp.apply(get_dom_visit,axis=1)

In [248]:
tmp.head()

,store,dom,dom_visits,mon_visitors,dom_pattern_train
0,0,0,84,3122,0.026906
1,0,1,113,3122,0.036195
2,0,2,118,3122,0.037796
3,0,3,136,3122,0.043562
4,0,4,87,3122,0.027867


In [249]:
together = pd.merge(together,tmp[['store','dom','dom_pattern_train']],how='left',on=['store','dom'])

In [250]:
together.shape

(284127, 27)

In [251]:
together.fillna(-1,inplace=True)

In [254]:
together.drop('dom_pattern_train_y',axis=1,inplace=True)

## another round of xgb test

In [255]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.25,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.497461333333
0.15 0.480895333333
0.2 0.480688


## day of year visit

In [256]:
tmp = together[together.year==0].groupby(['store','doy'],as_index=False
                                        )['visitors'].sum().rename(columns={'visitors':'doy_visits'})

In [257]:
tmp0 = together[together.year==0].groupby(['store'],as_index=False
                                         )['visitors'].sum().rename(columns={'visitors':'store_visits'})

In [258]:
tmp.shape

(174535, 3)

In [259]:
tmp0.shape

(819, 2)

In [260]:
tmp = pd.merge(tmp,tmp0,how='inner',on='store')

In [261]:
tmp.shape

(174535, 4)

In [262]:
tmp.head()

,store,doy,doy_visits,store_visits
0,0,182,14,3122
1,0,183,19,3122
2,0,184,17,3122
3,0,186,15,3122
4,0,187,15,3122


In [263]:
def get_doy_visit(data):
    if data.store_visits==0:
        return 0
    else:
        return data.doy_visits/data.store_visits

In [266]:
tmp['doy_pattern_2016'] = tmp.apply(get_doy_visit,axis=1)

In [267]:
tmp.head()

,store,doy,doy_visits,store_visits,dom_pattern_2016,doy_pattern_2016
0,0,182,14,3122,0.004484,0.004484
1,0,183,19,3122,0.006086,0.006086
2,0,184,17,3122,0.005445,0.005445
3,0,186,15,3122,0.004805,0.004805
4,0,187,15,3122,0.004805,0.004805


In [268]:
together = pd.merge(together,tmp[['store','doy','doy_pattern_2016']],how='left',on=['store','doy'])

In [269]:
together.shape

(284127, 27)

In [270]:
together.fillna(-1,inplace=True)

## another round of xgb test

In [273]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.5,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.318103
0.15 0.285632333333
0.2 0.281223333333
0.25 0.283435666667
0.3 0.287309
0.35 0.290908
0.4 0.295191666667
0.45 0.301708


In [275]:
for i in np.arange(50,500,50):
    param = {
        'eta':0.2,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,i,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

50 0.277734333333
100 0.279605666667
150 0.282549333333
200 0.284724
250 0.286304333333
300 0.287366333333
350 0.288110333333
400 0.288643
450 0.289021


## train 

In [277]:
test_id = together.id.apply(lambda x:True if x!=1.0 else False)

In [278]:
test_data = together[test_id]

In [279]:
test_data.shape

(32019, 27)

In [280]:
dtest = xgb.DMatrix(test_data.drop(['id','visitors','air_store_id','visit_date'],axis=1))

In [300]:
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date'],axis=1),train_data.visitors)

In [301]:
param = {
        'eta':0.2,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':50,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
model = xgb.train(param,dtrain)

In [302]:
preds = model.predict(dtest)

In [292]:
def retrive_y(data):
    return np.exp(data)-1

In [284]:
test_data.head()

,air_store_id,area,dom,doq,dow,doy,gen,holi,id,month,...,store_pop,store_genre_pop,genre_pop,store_pop_monthly,last_genre_pop,last_month_comp,last_quarter_comp,dow_pattern_2016,dom_pattern_train_x,doy_pattern_2016
2371,air_ba937bf13d40fb24,33,22,22,6,112,13,0,air_ba937bf13d40fb24_2017-04-23,3,...,0.030911,0.176627,0.175009,0.018037,0.168671,0.106936,0.113203,0.004939,0.032555,0.005604
2372,air_ba937bf13d40fb24,33,23,23,0,113,13,0,air_ba937bf13d40fb24_2017-04-24,3,...,0.030911,0.176627,0.175009,0.018037,0.168671,0.106936,0.113203,0.088011,0.031881,0.003923
2373,air_ba937bf13d40fb24,33,24,24,1,114,13,0,air_ba937bf13d40fb24_2017-04-25,3,...,0.030911,0.176627,0.175009,0.018037,0.168671,0.106936,0.113203,0.129322,0.037270,-1.000000
2374,air_ba937bf13d40fb24,33,25,25,2,115,13,0,air_ba937bf13d40fb24_2017-04-26,3,...,0.030911,0.176627,0.175009,0.018037,0.168671,0.106936,0.113203,0.171307,0.031881,0.004063
2375,air_ba937bf13d40fb24,33,26,26,3,116,13,0,air_ba937bf13d40fb24_2017-04-27,3,...,0.030911,0.176627,0.175009,0.018037,0.168671,0.106936,0.113203,0.148069,0.032443,0.002662


In [303]:
result = pd.concat([test_data[['id']].reset_index(drop=True),pd.Series(preds,name='visitors')],axis=1)

In [304]:
result.head()

,id,visitors
0,air_ba937bf13d40fb24_2017-04-23,-0.461857
1,air_ba937bf13d40fb24_2017-04-24,0.618590
2,air_ba937bf13d40fb24_2017-04-25,3.138249
3,air_ba937bf13d40fb24_2017-04-26,5.638799
4,air_ba937bf13d40fb24_2017-04-27,3.311872


In [307]:
def no_negative(data):
    if data.visitors<0:
        return 0
    else:
        return data.visitors

In [308]:
result['visitors'] = result.apply(no_negative,axis=1)

In [309]:
result.head()

,id,visitors
0,air_ba937bf13d40fb24_2017-04-23,0.000000
1,air_ba937bf13d40fb24_2017-04-24,0.618590
2,air_ba937bf13d40fb24_2017-04-25,3.138249
3,air_ba937bf13d40fb24_2017-04-26,5.638799
4,air_ba937bf13d40fb24_2017-04-27,3.311872


In [305]:
result.visitors.max()

37.614086

In [291]:
np.exp(max(preds))-1

1.968386173248291

In [296]:
result.visitors.max()

1.9683862

In [297]:
just_see = model.predict(dtrain)

In [299]:
np.exp(just_see.max())-1

76.793754577636719

In [310]:
result.to_csv('sub2.csv',index=False)

## back to digging

In [311]:
together.columns

Index(['air_store_id', 'area', 'dom', 'doq', 'dow', 'doy', 'gen', 'holi', 'id',
       'month', 'quarter', 'store', 'visit_date', 'visitors', 'year',
       'last_quarter_gen_pop', 'last_quarter_s_pop', 'store_pop',
       'store_genre_pop', 'genre_pop', 'store_pop_monthly', 'last_genre_pop',
       'last_month_comp', 'last_quarter_comp', 'dow_pattern_2016',
       'dom_pattern_train_x', 'doy_pattern_2016'],
      dtype='object')

In [313]:
train_data = together[train_id]
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date','doy_pattern_2016',
                                     ],axis=1),train_data.visitors.apply(np.log1p))
for i in np.arange(0.1,0.5,0.05):
    param = {
        'eta':i,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':100,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
    hist = xgb.cv(param,dtrain,30,metrics='rmse')
    print(i,hist['test-rmse-mean'].iloc[-1])

0.1 0.497461333333
0.15 0.480895333333
0.2 0.480688
0.25 0.482984666667
0.3 0.487492666667
0.35 0.492163666667
0.4 0.497231
0.45 0.503901666667


In [316]:
together.columns

Index(['air_store_id', 'area', 'dom', 'doq', 'dow', 'doy', 'gen', 'holi', 'id',
       'month', 'quarter', 'store', 'visit_date', 'visitors', 'year',
       'last_quarter_gen_pop', 'last_quarter_s_pop', 'store_pop',
       'store_genre_pop', 'genre_pop', 'store_pop_monthly', 'last_genre_pop',
       'last_month_comp', 'last_quarter_comp', 'dow_pattern_2016',
       'dom_pattern_train_x', 'doy_pattern_2016'],
      dtype='object')

In [329]:
dtest = xgb.DMatrix(test_data.drop(['id','visitors','air_store_id','visit_date','doy_pattern_2016',
                                   'dom_pattern_train_x','dow_pattern_2016',
                                   'store_genre_pop', 'genre_pop', 'store_pop_monthly',
                                   'last_genre_pop',
       'last_month_comp', 'last_quarter_comp'],axis=1))
dtrain = xgb.DMatrix(train_data.drop(['id','visitors','air_store_id','visit_date','doy_pattern_2016',
                                     'dom_pattern_train_x','dow_pattern_2016',
                                     'store_genre_pop', 'genre_pop', 
                                      'store_pop_monthly',
                                     'last_genre_pop',
       'last_month_comp', 'last_quarter_comp'],axis=1),train_data.visitors)

In [330]:
param = {
        'eta':0.2,'max_depth':16,'subsample':0.94,'colsample_bytree':0.73,'lamdba':0.3,'alpha':0.8,\
        'min_child_weight':29,'num_boost_round':50,'objective':'reg:linear','booster':'gbtree',\
        'tree_method':'hist','max_bin':150
    }
model = xgb.train(param,dtrain)
preds = model.predict(dtest)
result = pd.concat([test_data[['id']].reset_index(drop=True),pd.Series(preds,name='visitors')],axis=1)
result.head()

,id,visitors
0,air_ba937bf13d40fb24_2017-04-23,-1.118583
1,air_ba937bf13d40fb24_2017-04-24,-0.073194
2,air_ba937bf13d40fb24_2017-04-25,-0.069514
3,air_ba937bf13d40fb24_2017-04-26,-0.103742
4,air_ba937bf13d40fb24_2017-04-27,-0.109151


In [331]:
result[result.id=='air_00a91d42b08b08d9_2017-04-23']

,id,visitors
4524,air_00a91d42b08b08d9_2017-04-23,0.334315


## merge with kernel data

In [332]:
kernel_train = pd.read_csv('kernel_train.csv')
kernel_test = pd.read_csv('kernel_test.csv')

In [333]:
kernel_train.head()

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,rv2_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,air_store_id2
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.84375,...,-1.0,air_ba937bf13d40fb24_2016-01-13,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,599
1,air_ba937bf13d40fb24,2016-01-20,31,2,2016,1,6,0,7.0,23.84375,...,-1.0,air_ba937bf13d40fb24_2016-01-20,-1.0,-1.0,-1.0,20160120,8.362564,4.521799,175.409667,599
2,air_ba937bf13d40fb24,2016-01-27,24,2,2016,1,6,0,7.0,23.84375,...,-1.0,air_ba937bf13d40fb24_2016-01-27,-1.0,-1.0,-1.0,20160127,8.362564,4.521799,175.409667,599
3,air_ba937bf13d40fb24,2016-02-03,18,2,2016,2,6,0,7.0,23.84375,...,-1.0,air_ba937bf13d40fb24_2016-02-03,-1.0,-1.0,-1.0,20160203,8.362564,4.521799,175.409667,599
4,air_ba937bf13d40fb24,2016-02-10,32,2,2016,2,6,0,7.0,23.84375,...,-1.0,air_ba937bf13d40fb24_2016-02-10,-1.0,-1.0,-1.0,20160210,8.362564,4.521799,175.409667,599


In [334]:
kernel_train.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'year', 'month',
       'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors',
       'median_visitors', 'max_visitors', 'count_observations',
       'air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'air_genre_name0', 'air_area_name0', 'air_genre_name1',
       'air_area_name1', 'air_genre_name2', 'air_area_name2',
       'air_genre_name3', 'air_area_name3', 'air_genre_name4',
       'air_area_name4', 'air_genre_name5', 'air_area_name5',
       'air_genre_name6', 'air_area_name6', 'air_genre_name7',
       'air_area_name7', 'air_genre_name8', 'air_area_name8',
       'air_genre_name9', 'air_area_name9', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x',
       'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int',
       'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2'],
      dtype='object')

In [335]:
col_to_drop = ['air_genre_name', 'air_area_name', 'latitude', 'longitude',
       'air_genre_name0', 'air_area_name0', 'air_genre_name1',
       'air_area_name1', 'air_genre_name2', 'air_area_name2',
       'air_genre_name3', 'air_area_name3', 'air_genre_name4',
       'air_area_name4', 'air_genre_name5', 'air_area_name5',
       'air_genre_name6', 'air_area_name6', 'air_genre_name7',
       'air_area_name7', 'air_genre_name8', 'air_area_name8',
       'air_genre_name9', 'air_area_name9', 'dow', 'year', 'month',
       'day_of_week', 'holiday_flg', 'air_store_id2']

In [336]:
kernel_train.drop(col_to_drop,axis=1,inplace=True)
kernel_test.drop(col_to_drop,axis=1,inplace=True)

In [337]:
kernel_train.shape

(250468, 24)

In [338]:
kernel_train.columns

Index(['air_store_id', 'visit_date', 'visitors', 'min_visitors',
       'mean_visitors', 'median_visitors', 'max_visitors',
       'count_observations', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y',
       'rv1_y', 'rs2_y', 'rv2_y', 'id', 'total_reserv_sum',
       'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int',
       'var_max_lat', 'var_max_long', 'lon_plus_lat'],
      dtype='object')

In [339]:
together.columns

Index(['air_store_id', 'area', 'dom', 'doq', 'dow', 'doy', 'gen', 'holi', 'id',
       'month', 'quarter', 'store', 'visit_date', 'visitors', 'year',
       'last_quarter_gen_pop', 'last_quarter_s_pop', 'store_pop',
       'store_genre_pop', 'genre_pop', 'store_pop_monthly', 'last_genre_pop',
       'last_month_comp', 'last_quarter_comp', 'dow_pattern_2016',
       'dom_pattern_train_x', 'doy_pattern_2016'],
      dtype='object')

In [341]:
final_train = pd.merge(kernel_train,together.drop('id',axis=1),how='left',on=['air_store_id','visit_date'])

In [342]:
final_test = pd.merge(kernel_test,together.drop('id',axis=1),how='left',on=['air_store_id','visit_date'])

In [2]:
final_train.to_csv('last_train.csv',index=False)
final_test.to_csv('last_test.csv',index=False)

NameError: name 'final_train' is not defined

In [344]:
final_train.shape

(250468, 48)

In [345]:
kernel_train.shape

(250468, 24)